## Create Spark Session

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

## Some Setup (not needed in DSW)

In [ ]:
# Append system path
import sys
sys.path.append('../')

import os
# Tunnel Muttley for production WFO GRPC
# os.system('ssh -MfN -L 5435:localhost:5435 adhoc20-dca1')
# Or start WFO service locally.

## Initialize MaLearn in lazy mode
We need to find out a way to package operators fat jar before running eager mode on local.

In [ ]:
import malearn
from malearn import MaLearnConfig

config = MaLearnConfig()
config.eager_mode_enabled = False
config.workflow_service_port = 9877
malearn.init(config)

## Use project API

In [ ]:
from malearn.project import MichelangeloProject
project = MichelangeloProject(project_id='Jin_test_non_secure')
df = project.read_data_source('integration_test_datasource')
sink_op = df.to_hive_table('tmp.michelangelo_test', drop_existing_table=True, ttl=1, _return_op=True)

## Submit workflow to WFO

In [ ]:
from malearn import Session
with Session(project_id='Jin_test_non_secure') as sess:
    sess._config.workflow_service_config.port = 5435
    sess.queue = 'michelangelo-dev'
    sess.run(sink_op)
    print(sess.workflow.format_logical())